# <center>FIT5196- Assignment 1: Parsing Data and Text Preprocessing.</center>
## <center>Task 1: Parsing Data</center>
### Student ID: 29592461
### Student Name: Dinesh Karthikeyan
#### Language: Python 3.7.0
#### Working Environment: Jupyter Notebook
#### Packages Used: re,  json
#### Regex Checker: Pythex.
#### Overview of  methodology: 
* The data set to be parsed is Unit guide (in html format) for several units, which is to be parsed and converted to xml and json in the format specified in the specification.
* The required fields Unit Code, Title, Prerequistite, Synopsis, Requirement, Prohibitions, Outcomes and Chief examiner for each unit are to be fetched and stored in xml and json file with the format specified.
* First step is to read the contents from the file "*29592461.txt*".
* Then using Regular expression take details of every unut separately and store it in a list.
* Proceed further and fetch all other required fields using regular expression by iterating through this list and writing the values into the xml and json files based on the format specified by the specification and add appropriate values if a unit doesn't have the required field or the value for the required field. (i.e.) For Prerequisite, Prohibitions, Requirement, Outcomes the value to be used is "**NA**" and for Chief Examiner it is "**TBA**".
* After parsing the required data, store the values in the specified format into "*29592461.xml*" for xml and "*29592461.json*" for json.

#### Import Section

* re - used for Regular expression functionalities.
* json - used for writing to Json.

In [1]:
import re
import json

#### Global Variable Intialisation

* Initialising variables for all the regular expressions (regex) to be used and string varialbles for json and xml strings that are to be written in the file as output.    

In [2]:
file=open('29592461.txt')
file_content=file.read()
json_dict={'units':{'unit':[]}}
i=-1
xml_op='<?xml version="1.0" encoding="UTF-8" ?>\n<units>\n'   

* For fetching each unit and to put into list the regex (Units): `((?<=<div class="hbk-banner-box">)([\w\W\s]*?)(?=</nav>|<!-- /.content_container--> </div>))` is used.
* The approach to get this regex : 
    * To fectch the contents between `<div class="hbk-banner-box">` and `</nav>` or `<!-- /.content_container--> </div>` tags, so these form the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
    * The second group is `[\w\W\s]*` fetch all the words,symbols and spaces, this will bring entire file so used `?` for this group to get values in non greedy manner and all three sub groups are put into a major group.

In [3]:

units_pattern=r'((?<=<div class="hbk-banner-box">)([\w\W\s]*?)(?=</nav>|<!-- /.content_container--> </div>))'      

* For fetching Unit code from the unit details (fetched with above regex (Units) and put into list) the regex: `((?<=<span class="unitcode">)(\w{3,4}\d{4})(?=</span>))` is used.
    * The approach to get this regex:
        * To fetch the contents between `<span class="unitcode">` and `</span>` tags, so these for the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
        * The second group is `\w{3,4}\d{4}` fetch a string having 3 or 4 alphabet and 4 digits (e.g. FIT5196, ABCD1000). 
        * All three groups into a major group so as to get only the matched unitcode.

In [4]:
unit_code_pattern = r'((?<=<span class="unitcode">)(\w{3,4}\d{4})(?=</span>))'


* For fetching Synopsis from the unit details (fetched with above regex (Units) and put into list) the regex: `((?<=<h2 class="hbk-heading">Synopsis</h2>\s<div>\s<p>)(.*)(?=</p>\s))` is used.
    * The approach to get this regex:
        * To fetch the contents between `<h2 class="hbk-heading">Synopsis</h2>\s<div>\s<p>` and `</p>\s` tags, so these for the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
        * The second group is `.*` fetch all the data. 
        * All three groups into a major group so as to get the synopsis (only within the first p tag).

In [5]:
unit_synopsis_pattern=r'((?<=<h2 class="hbk-heading">Synopsis</h2>\s<div>\s<p>)(.*)(?=</p>\s))'


* For fetching Prerequisite and Corequisite from the unit details (fetched with above regex (Units) and put into list) the regex: `(((?:<p class="hbk-preamble-heading">Prerequisites</p>\s)(.*)(?:\s<p class="hbk-preamble-heading">Co-requisites</p>\s(.*))?)|((?:\s<p class="hbk-preamble-heading">Co-requisites</p>\s(.*))(?:</span>|</p>)))` is used.
    * The approach to get this regex:
        * To fetch contents starting with `<p class="hbk-preamble-heading">Prerequisites</p>\s)` or` \s<p class="hbk-preamble-heading">Co-requisites</p>\s` and ends with `</span>|</p>`. But in few cases either prequisite and corequisite both are present or only one of them is present so `(<p class="hbk-preamble-heading">Prerequisites</p>\s)(.*)(?:\s<p class="hbk-preamble-heading">Co-requisites</p>\s(.*))?` this to get prerequiste and corequisite if both are present or only prerequisite alone if no corequiste, `(<p class="hbk-preamble-heading">Prerequisites</p>\s)(.*)(?:\s<p class="hbk-preamble-heading">Co-requisites</p>\s(.*))?)|((\s<p class="hbk-preamble-heading">Co-requisites</p>\s(.*))` this is used to fetch even prerequisite and corequiste even if the other is missng.
        * `?:` is added to make the groups non capturing so as to get only the values between them.
        * Finally all the groups are put into a major group to fetch only the values required.

In [6]:
unit_Prerequisite_pattern=r'(((?:<p class="hbk-preamble-heading">Prerequisites</p>\s)(.*)(?:\s<p class="hbk-preamble-heading">Co-requisites</p>\s(.*))?)|((?:\s<p class="hbk-preamble-heading">Co-requisites</p>\s(.*))(?:</span>|</p>)))'


* For fetching Requirement from the unit details (fetched with above regex (Units) and put into list) the regex: `((?<=<h2 class="hbk-heading">Assessment</h2>\s<div>)([\w\W\s]*?)(?=</div>))` is used.
    * The approach to get this regex:
        * To fetch the contents between `<h2 class="hbk-heading">Assessment</h2>\s<div>` and `</div>` tags, so these for the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
        * The second group is `[\w\W\s]*` fetch all the words, symbols and spaces,this will bring entire file so used `?` for this group to get values in non greedy manner and all three sub groups are put into a major group.

In [7]:
unit_requirement_pattern=r'((?<=<h2 class="hbk-heading">Assessment</h2>\s<div>)([\w\W\s]*?)(?=</div>))'


* For fetching Prohibitions from the unit details (fetched with above regex (Units) and put into list) the regex: `((?<=<p class="hbk-preamble-heading">Prohibitions</p>\s<p>)([\w\W\s]*?)(?=</p>\s</div>))` is used.
    * The approach to get this regex:
        * To fetch the contents between `<p class="hbk-preamble-heading">Prohibitions</p>\s<p>` and `</p>\s</div>` tags, so these for the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
        * The second group is `[\w\W\s]*` fetch all the words, symbols and spaces,this will bring entire file so used `?` for this group to get values in non greedy manner and all three sub groups are put into a major group.

In [8]:
unit_prohibition_pattern=r'((?<=<p class="hbk-preamble-heading">Prohibitions</p>\s<p>)([\w\W\s]*?)(?=</p>\s</div>))'

* For fetching Unit Title from the unit details (fetched with above regex (Units) and put into list) the regex: `(?<=</span>\s-\s)(.*)(?=<span)` is used.
    * The approach to get this regex:
        * To fetch the contents between `</span>\s-\s` and `<span` tags, so these for the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
        * The second group is `.*` fetch all the data. 
        * All three groups into a major group so as to get the title.

In [9]:
unit_title_pattern=r'(?<=</span>\s-\s)(.*)(?=<span)'


* For fetching Outcomes from the unit details (fetched with above regex (Units) and put into list) the regex: `((?<=<h2 class="hbk-heading">Outcomes</h2>\s)([\w\W\s]*?)(?=((</ol>\s</div>|</ul>\s</div>))))` is used.
    * The approach to get this regex:
        * To fetch the contents between `<h2 class="hbk-heading">Outcomes</h2>\s` and `((</ol>\s</div>|</ul>\s</div>)))` tags, so these for the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
        * The second group is `[\w\W\s]*` fetch all the words, symbols and spaces,this will bring entire file so used `?` for this group to get values in non greedy manner and all three sub groups are put into a major group.
        * This gives all the Vslues under the Outcome tag.
        * From that `(?:<li>)([\W\w]*?)(?:</li>\s)` this is used to get only the outcome values that are under the lis tag.

In [10]:
unit_outcome_pattern=r'((?<=<h2 class="hbk-heading">Outcomes</h2>\s)([\w\W\s]*?)(?=((</ol>\s</div>|</ul>\s</div>))))'


In [11]:
unit_outcome_list_pattern=r'(?:<li>)([\W\w]*?)(?:</li>\s)'


* For fetching Chief-Examiner from the unit details (fetched with above regex (Units) and put into list) the regex: `((?<=<p class="hbk-highlight-heading">Chief examiner\(s\)</p>\s<p>\s)([\w\W\s]*?)(?=</p>))` is used.
    * The approach to get this regex:
        * To fetch the contents between `<p class="hbk-highlight-heading">Chief examiner\(s\)</p>\s<p>\s` and `</p>` tags, so these for the first and third group, to specify starts with `?<=` is added and to specify ends with `?=` is used.
        * The second group is `[\w\W\s]*` fetch all the words, symbols and spaces,this will bring entire file so used `?` for this group to get values in non greedy manner and all three sub groups are put into a major group.

In [12]:
unit_chief_examiner_pattern=r'((?<=<p class="hbk-highlight-heading">Chief examiner\(s\)</p>\s<p>\s)([\w\W\s]*?)(?=</p>))'


#### Parsing Phase:

* Fetching the Unit details and putting into the list (units).

In [13]:
units=re.findall(units_pattern,file_content)

* Iterating through the list to parse for each unit. (Inline comments gives the description )

In [14]:
for unit_val in units: # Traversing through each unit.
    unit_code=''
    unit_title=''
    unit_prerequisite=[]
    unit_synopsis=''
    unit_requirements=[]
    unit_prohibitions_list=[]
    unit_outcome=[]
    unit_chief_examiner=[]
    unit=unit_val[1]    
    # Fetching Unit title.
    unit_title=re.findall(unit_title_pattern,unit) 
    i+=1
    # Fetching Unit code.
    unit_code=re.findall(unit_code_pattern,unit)[0][1] if len(re.findall(unit_code_pattern,unit))>0 else 'NA' 
    unit_prerequisite_data=re.findall(unit_Prerequisite_pattern,unit) # Fetching the Prerequisite contents.
    if len(unit_prerequisite_data)>0:
        for unit_prerequisite_data_val in unit_prerequisite_data: 
            unit_prerequisite_val=unit_prerequisite_data_val[0]
            # Parsing through the Prerequisite contents to get only the unit codes of prerequisite.
            unit_prerequisite=list(set(re.findall(r'\w{3,4}\d{4}',unit_prerequisite_val)))
            
            if len(unit_prerequisite) ==0 :
                unit_prerequisite.append('NA')  # Adding NA for empty prerequisite.      
    else:
        unit_prerequisite.append('NA')  # Adding NA if there no Prerequisite tag.
 
    # Fetching Synopsis Contents.
    unit_synopsis_temp=re.findall(unit_synopsis_pattern,unit)[0][1] if len(re.findall(unit_synopsis_pattern,unit))>0 else 'NA'
    # Removing Unnecessary tags.
    unit_synopsistemp1=re.sub(r'((?:<span)(?:.*?)(?:html">))','',unit_synopsis_temp)
    unit_synopsistemp2=re.sub(r'(</a.</span>)','',unit_synopsistemp1)
    unit_synopsis=unit_synopsistemp2
    # Fetching Requirement Contents.
    unit_requirement=re.findall(unit_requirement_pattern,unit)
    if len(unit_requirement)>0:
        for unit_requirement_data in unit_requirement:   # Iterating throught to get the required conetnt. 
            # Fecthing the values from p tags.
            unit_requirement_val_list=re.findall(r'(((?<=<p>)([\w\W\s]*?)(?=</p>)))',unit_requirement_data[1])            
            if len(unit_requirement_val_list)==0:
                unit_requirements.append('NA') # Adding NA if no matches.
            else:    
                for unit_requirement_val in unit_requirement_val_list: # Iterate Matches  and remove unnecessary tags.
                        temp= re.sub(r'((?:<a)(?:[\w\W\s]*?)(?:.html">))', '', unit_requirement_val[1])
                         
                        temp1=re.sub(r'((?:</a><span)(?:.*)(?:">))|((?:<span)(?:.*)(?:">))','',temp)
                        
                        
                        temp2=re.sub(r'(</a>|</span>)','',temp1)
                        unit_requirements.append(temp2)
                        #unit_requirements.append(unit_requirement_val[1])
                        
    else:
        unit_requirements.append('NA') # Add NA if no Tag found.
            
    
    unit_prohibitions_data=re.findall(unit_prohibition_pattern,unit) # Fetching Prohibitions content.
    
    
    if len(unit_prohibitions_data)==0:
        unit_prohibitions_list.append('NA') # Adding NA if no matches.
    else:
        for unit_prohibition in unit_prohibitions_data: # Iterate through matches and fetch only unit codes.
            unit_prohibitions_list_temp=(re.findall(r'\w{3,4}\d{4}',unit_prohibition[1]))
            # Add NA if no unit code is found  else Add the unique values of unit code.
            unit_prohibitions_list= list(set(unit_prohibitions_list+ unit_prohibitions_list_temp if len(unit_prohibitions_list_temp)>0 else ['NA']))
    
    unit_outcomes=re.findall(unit_outcome_pattern,unit) # Fetching Outcome Contents.
    if len(unit_outcomes)>0:
        for unit_outcome_val in unit_outcomes: # Iterate through the matches to fetch the list (html li tag) values.
            temp= re.findall(unit_outcome_list_pattern,unit_outcome_val[1]) # Fetching values in the list (html li tag)
            for each in temp:
                # Removing Unnecessary tags.
                temp1=re.sub('((<span)(.*?)(html">))|((<ol)(.*?)(">))','',each)
                temp2=re.sub(r'(</a></span>)','',temp1)
                temp3=re.sub(r'(<p>|</p>|<ul>|<li>)','',temp2)
                unit_outcome.append(temp3)
            
    else:
        unit_outcome.append("NA") # Adding NA if No tag.
    # Fetching Chief examiner contents.
    unit_chief_examiner_data=re.findall(unit_chief_examiner_pattern,unit) if len(re.findall(unit_chief_examiner_pattern,unit))>0 else ['TBA']
    # If the Value is not TBA  directly check for the inner content and if its TBA add TBA 
    # else add the values matching a pattern.
    if unit_chief_examiner_data[0]!='TBA':
        if "TBA" in str(unit_chief_examiner_data[0]):
            unit_chief_examiner.append("TBA")
            
        else:
            unit_chief_examiner=re.findall(r'(?:(?:<a.*">)([\w\W\s]*?)(?:</a>))|((?:A/)(.*))',unit_chief_examiner_data[0][1])
    else:
        unit_chief_examiner.append("TBA") # Add TBA if it has TBA.
    
    # Xml string and Json dictionary formation for writing into file.
    xml_op+="<unit id='{0}'>\n\n\n<title>{1}</title>\n<synopsis>{2}</synopsis>\n<pre_requistics>".format(unit_code,unit_title[0],unit_synopsis)
    
    json_dict['units']['unit'].append({'@id':unit_code,'title':unit_title[0],'synopsis':unit_synopsis})
    if unit_prerequisite[0]!="NA":
        if len(unit_prerequisite)>1:
            json_dict['units']['unit'][i]['pre_requistics']={'pre_requistic':[]}
        elif len(unit_prerequisite)==1:
            json_dict['units']['unit'][i]['pre_requistics']={'pre_requistic':''}
        xml_op+="\n"
    for each in unit_prerequisite:
        if each!="NA":
            xml_op+="<pre_requistic>{0}</pre_requistic>".format(each)
            if len(unit_prerequisite)>1:
                json_dict['units']['unit'][i]['pre_requistics']['pre_requistic'].append(each)
            elif len(unit_prerequisite)==1:
                json_dict['units']['unit'][i]['pre_requistics']['pre_requistic']=each
        else:
            xml_op+=each
            json_dict['units']['unit'][i]['pre_requistics']=each
                
    xml_op+="</pre_requistics>\n<prohibisions>"
    count=0
    for each in unit_prohibitions_list:
        count+=1
        if each=="NA":
            xml_op+="{0}</prohibisions>\n".format(each)
            json_dict['units']['unit'][i]['prohibisions']=each
        else:
            if count==1:
                xml_op+="\n"
                if len(unit_prohibitions_list)>1:
                    json_dict['units']['unit'][i]['prohibisions']={'prohibision':[]}
                    
                elif len(unit_prohibitions_list)==1:
                    json_dict['units']['unit'][i]['prohibisions']={'prohibision':''}
            if len(unit_prohibitions_list)>1:
                json_dict['units']['unit'][i]['prohibisions']['prohibision'].append(each)
            elif len(unit_prohibitions_list)==1:
                json_dict['units']['unit'][i]['prohibisions']['prohibision']=each
            xml_op+="<prohibision>{0}</prohibision>".format(each)
            
            if count==len(unit_prohibitions_list):
                xml_op+="</prohibisions>\n"
    xml_op+="<requirements>"
    
    new=0
    for each in unit_requirements:
        new+=1
        if each=="NA":
            xml_op+="NA"
            json_dict['units']['unit'][i]['requirements']="NA"
        else:
            if new==1:
                xml_op+="\n"
                
                if len(unit_requirements)>1:
                    json_dict['units']['unit'][i]['requirements']={'requirement':[]}
                    
                elif len(unit_requirements)==1:
                    json_dict['units']['unit'][i]['requirements']={'requirement':''}
            if len(unit_requirements)>1:
                json_dict['units']['unit'][i]['requirements']['requirement'].append(each)
            elif len(unit_requirements)==1:
                json_dict['units']['unit'][i]['requirements']['requirement']=each  
            xml_op+="<requirement>{0}</requirement>".format(each)
              
    xml_op+="</requirements>\n<outcomes>\n"
    new=0
    for each in unit_outcome:
        new+=1
        # Using flag (new) for checing first value and index for units (i).
        if new==1:
            if len(unit_outcome)>1:
                json_dict['units']['unit'][i]['outcomes']={'outcome':[]}
                
            elif len(unit_outcome)==1:
                json_dict['units']['unit'][i]['outcomes']={'outcome':''}
        if len(unit_outcome)>1:        
            json_dict['units']['unit'][i]['outcomes']['outcome'].append(each)
        elif len(unit_outcome)==1:
            json_dict['units']['unit'][i]['outcomes']['outcome']=each
            
        xml_op+="<outcome>{0}</outcome>".format(each)
       
    xml_op+="</outcomes>\n<chief_examiners>"
   
    if unit_chief_examiner[0]!="TBA":
        xml_op+="\n"
        if len(unit_chief_examiner)>1:
            json_dict['units']['unit'][i]['chief_examiners']={'chief_examiner':[]}
        elif len(unit_chief_examiner)==1:
            json_dict['units']['unit'][i]['chief_examiners']={'chief_examiner':''}
        for each in unit_chief_examiner:
            if each[0]!="TBA" and each[0] is not None:
                xml_op+="<chief_examiner>{0}</chief_examiner>".format(each[0])
                if len(unit_chief_examiner)==1:
                    json_dict['units']['unit'][i]['chief_examiners']['chief_examiner']=each[0]
                elif len(unit_chief_examiner)>1:
                    json_dict['units']['unit'][i]['chief_examiners']['chief_examiner'].append(each[0])
            elif each[1]!="TBA" and each[1] is not None:
                xml_op+="<chief_examiner>{0}</chief_examiner>".format(each[1])
                if len(unit_chief_examiner)==1:
                    json_dict['units']['unit'][i]['chief_examiners']['chief_examiner']=each[1]
                elif len(unit_chief_examiner)>1:
                    json_dict['units']['unit'][i]['chief_examiners']['chief_examiner'].append(each[1])
    else:
        xml_op+="TBA"
        json_dict['units']['unit'][i]['chief_examiners']="TBA"
            
    xml_op+="</chief_examiners>\n</unit>\n"

#### Writing output:
* Xml string is constructed with all the values from the parsing phase and with the necessary fromating.
* Json dictionary is formed with the values from the parsing phase and with the dumps() function dumped into a string.
* Both strings are written in separate files *2959246951.xml* and *29592461.json* respectively.

In [15]:
# Writing the XMl String with the required formats into 29592461.xml.
xml_op+="</units>\n" 
xml_op=xml_op.replace("&amp","&#38")
output=open("29592461.xml",'w')
output.write(xml_op)
output.close()
# Dumping json dictionary into json string and writing into 29592461.json file.
json_string=json.dumps(json_dict,indent=4)
json_output=open("29592461.json","w")
# Replacing the values according to unicode.
json_string=json_string.replace("&amp;","&") 
json_string=json_string.replace("&gt;",">")
json_output.write(json_string)
json_output.close()